# Module 2 Assignment

### Use MySQL to create a database and store data from a datafile.  Show an example query from the new database.

In [2]:
# Library Imports

import getpass # Allows for Hidden Password Input
import mysql.connector as mysql # Communication to MySQL Server
import pandas as pd
import numpy as np

#### I created two classes for Database and Table and created methods for verfying, creating, and displaying using the MySQL Connector Library

In [3]:
# Class for Database

class Database:
    def __init__(self, name, user, passwd):
        self.name = name
        self.user = user
        self.passwd = passwd
        self.host = "localhost"
        self.establish_connection(self.host, self.user, self.passwd)

    # Method to Verify Connection to MySQL Server and Executes DB Verification Method
    def establish_connection(self, host_name, user_name, password_provided):
        try:
            with mysql.connect(
                host = host_name,
                user = user_name,
                passwd = password_provided 
            ) as connection:
                self.verify_db_exists(connection)
        except mysql.Error as e:
            print(e)
        
    # Method to Create a New DB
    def create_db(self, connection):
        db = connection

        cursor = db.cursor()
        # SQL to Create Database
        sql = "CREATE DATABASE " + self.name
        cursor.execute(sql)
        cursor.close()
    
    # Method to manipulate database string returned via SQL
    def database_string_remove_characters(self, database):
        return  str(database).replace('\'','').replace('(','').replace(')','').replace(',','')

    # Method to Verify a DB already Exists with a specified name
    def verify_db_exists(self, connection):
        # Identify All DB Names
        cursor = connection.cursor()
        sql ="SHOW DATABASES"
        cursor.execute(sql)
        databases = cursor.fetchall()
        cursor.close()
        counter = 0
    
        # Isolate DB Name and Check against passed value
        for database in databases:
            new_db_string = self.database_string_remove_characters(database)
            if new_db_string == self.name:
                counter = 1
    
        # Create DB if Does Not Exist
        if counter != 1:
            self.create_db(connection)

        connection.close()
    
    # Method to connect to MySQL Server and Print all Available Databases
    def show_databases(self):
        db = mysql.connect(
            host = self.host,
            user = self.user,
            passwd = self.passwd,
        )
        
        cursor = db.cursor()
        sql = "SHOW DATABASES"
        cursor.execute(sql)
        databases = cursor.fetchall()
        cursor.close()

        for database in databases:
            print(self.database_string_remove_characters(database))

    # Method to connect to specified DB and Initialize it
    def connect_to_db(self):
        db = mysql.connect(
            host = self.host,
            user = self.user,
            passwd = self.passwd,
            database = self.name
        )
        return db

In [4]:
# Class for Tables inside a Database

class Table:
    def __init__(self, name, database):
        self.name = name
        self.db = database
        self.verify_table_exists()

    # Method to manipulate database string returned via SQL
    def table_string_remove_characters(self, table):
        return  str(table).replace('\'','').replace('(','').replace(')','').replace(',','')

    # Method to Verify a Table already Exists with a specified name
    def verify_table_exists(self):
        # Identify All DB Names
        cursor = self.db.cursor()
        sql = "SHOW TABLES"
        cursor.execute(sql)
        tables = cursor.fetchall()
        cursor.close()
        counter = 0
    
        # Isolate Table Name and Check against passed value
        for table in tables:
            new_table_string = self.table_string_remove_characters(table)
            if new_table_string == self.name.lower():
                counter = 1
    
        # Create Table if Does Not Exist
        if counter != 1:
            self.create_table()


    # Method to Create a New DB Table
    def create_table(self):
        cursor = self.db.cursor()
        # SQL to Create Database
        sql = "CREATE TABLE " + self.name + " (Year INTEGER, Total_mean FLOAT(10, 1), Total_Population FLOAT(10, 1), Total_Std_Err INTEGER, No_HS_Mean FLOAT(10, 1), No_HS_Population FLOAT(10, 1), No_HS_Std_Err INTEGER, HS_Mean FLOAT(10, 1), HS_Population FLOAT(10, 1), HS_Std_Err INTEGER, Some_Clg_AD_Mean FLOAT(10, 1), Some_clg_AD_Population FLOAT(10, 1), Some_Clg_AD_Std_Err INTEGER , BD_Mean FLOAT(10, 1), BD_Population FLOAT(10, 1), BD_Std_Err INTEGER, Grad_Mean FLOAT(10, 1), Grad_Population FLOAT(10, 1), Grad_Std_err INTEGER)"
        cursor.execute(sql)
        cursor.close()


    # Method to Print all available Tables
    def show_tables(self):
        cursor = self.db.cursor()
        sql = "SHOW TABLES"
        cursor.execute(sql)
        tables = cursor.fetchall()
        cursor.close()
        print(tables)
    
    # Method for inserting data into a table
    def data_insert(self,df):
        cursor = self.db.cursor()

        cols = "`,`".join([str(i) for i in df.columns.tolist()])

        for i,row in df.iterrows():
            sql = "INSERT INTO `" + self.name + "` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
            cursor.execute(sql, tuple(row))
            
        cursor.close()
        self.db.commit()

#### The following function strips the data from the provided Excel spreadsheet and assigns it to a pandas dataframe.

In [5]:
# Function to Extract Data from the provided Excel File into a Pandas DataFrame
def extract_excel_data():
    df = pd.read_excel('StevenHunt_Mod2_Data/Total Education and Mean Income By year.xlsx')
    return df


#### The following two functions generate the database and table object and inserts the data into the table in MySQL

In [6]:
def build_database_object():

    # Get User Input for Server Connection
    user = input('Enter Username: ')
    passwd = getpass.getpass('Enter Password: ')
    

    # Buil Database Object
    database = Database('education_census_data', user, passwd)
    
    '''database.show_databases()''' # Uncomment to return list of databases available in MySQL Server
    
     # Connect to Database
    return database.connect_to_db() 

def build_table_object(table_name, db_connection, df):
    # Create Table Object
    total_table = Table(table_name, db_connection)
    
    '''total_table.show_tables()''' # Uncomment to return list of tables available in the Database Object

    # Insert Data into Table
    total_table.data_insert(df)

#### Below is the Main function that runs all of the previous functions and prints the top 10 rows of the newly created table to show that the table was created correctly.

In [10]:
def main():
    # Build Database Object
    connection_to_db = build_database_object()
    
    # Get Dataframe from Excel File
    df = extract_excel_data()
    
    table_name = 'Total_Race_Sex'
    # Build Table Object
    build_table_object(table_name, connection_to_db, df)

    # Example Queries
    cursor = connection_to_db.cursor()
    sql = "SELECT * FROM " + table_name
    cursor.execute(sql)
    results = cursor.fetchall()
    cursor.close()
    
    # Show Top 10 Rows from Table
    counter = 0
    print("\nHere are the first 10 rows from the table that was just created:\n")
    for result in results:
        if counter <= 9:
            print(result)
            counter += 1


    # Close Database Connection
    connection_to_db.close()

   
if __name__ == '__main__':
    main()

Enter Username: Steve
Enter Password: ········

Here are the first 10 rows from the table that was just created:

(2019, 58544.0, 167215.0, 329, 29278.0, 11413.0, 552, 39371.0, 42598.0, 329, 45091.0, 46885.0, 411, 73163.0, 42153.0, 719, 106766.0, 24164.0, 1440)
(2018, 55619.0, 165179.0, 296, 27037.0, 12058.0, 481, 38936.0, 42882.0, 376, 43053.0, 46887.0, 416, 71155.0, 40231.0, 676, 99919.0, 23118.0, 1186)
(2017, 53536.0, 163871.0, 218, 26832.0, 12240.0, 383, 38145.0, 42816.0, 320, 41507.0, 47382.0, 291, 67763.0, 39153.0, 462, 98369.0, 22277.0, 933)
(2016, 51893.0, 162218.0, 217, 27800.0, 12281.0, 559, 36702.0, 42897.0, 296, 40201.0, 48128.0, 277, 67267.0, 37272.0, 508, 95203.0, 21639.0, 909)
(2015, 49994.0, 161074.0, 209, 25315.0, 13159.0, 422, 35615.0, 42404.0, 271, 38943.0, 47961.0, 261, 65482.0, 36348.0, 516, 92525.0, 21199.0, 898)
(2014, 47653.0, 158000.0, 196, 25236.0, 13197.0, 494, 34099.0, 42529.0, 240, 37945.0, 47023.0, 283, 62466.0, 35305.0, 452, 88056.0, 19944.0, 855)
(2013, 